In [1]:

from torch import __version__ as torch_version
print(torch_version)

from mmcv import __version__ as mmcv_version
print(mmcv_version)

from mmdet import __version__ as mmdet_version
print(mmdet_version)

from mmyolo import __version__
print(__version__)


2.0.0+cu118
2.0.1
3.3.0
0.6.0


In [2]:
import os 
os.chdir ("/workspace/objectdet")

In [3]:
from mmengine.config import Config

cfg = Config.fromfile("mmyolo/configs/yolov5/yolov5_n-v61_syncbn_fast_8xb16-300e_coco.py")
cfg['data_root'] = "datasets/Box-1/"

In [4]:
cfg.train_batch_per_gpu = 8

# cfg.data_root = "dataset/Cardboard-3/"

cfg.train_ann_file = 'train/_annotations.coco.json'
cfg.train_data_prefix = 'train/'  # Prefix of train image path
# Path of val annotation file
cfg.val_ann_file = 'valid/_annotations.coco.json'
cfg.val_data_prefix = 'valid/'  # Prefix of val image path

cfg.num_classes = 1  # Number of classes for classification

cfg.optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=dict(
        type='SGD',
        lr=2e-1,
        momentum=0.937,
        weight_decay=cfg.weight_decay,
        nesterov=True,
        batch_size_per_gpu=cfg.train_batch_size_per_gpu),
        constructor='YOLOv5OptimizerConstructor')

cfg.default_hooks = dict(
    param_scheduler=dict(
        type='YOLOv5ParamSchedulerHook',
        scheduler_type='linear',
        lr_factor=cfg.lr_factor,
        max_epochs=50),
    checkpoint=dict(
        type='CheckpointHook',
        interval=10,
        save_best='auto',
        max_keep_ckpts=3))

cfg.model.bbox_head.head_module.num_classes = cfg.num_classes
cfg.model.bbox_head.loss_cls.loss_weight = cfg.loss_cls_weight * (cfg.num_classes / 80 * 3 / cfg.num_det_layers)

metainfo = dict(
    classes = ('Box',),
    palette = [
        (220, 20, 60),
    ]
)

cfg.train_dataloader.batch_size = 8
cfg.train_dataloader.num_workers = 2
cfg.train_dataloader.dataset = dict(
     type=cfg.dataset_type,
        data_root=cfg.data_root,
        metainfo=metainfo,
        ann_file=cfg.train_ann_file,
        data_prefix=dict(img=cfg.train_data_prefix),
        filter_cfg=dict(filter_empty_gt=False, min_size=32),
        pipeline=cfg.train_pipeline
)

cfg.val_dataloader.batch_size = 1
cfg.val_dataloader.num_workers = 2
cfg.val_dataloader.dataset = dict(
        type=cfg.dataset_type,
        data_root=cfg.data_root,
        metainfo=metainfo,
        test_mode=True,
        data_prefix=dict(img=cfg.val_data_prefix),
        ann_file=cfg.val_ann_file,
        pipeline=cfg.test_pipeline,
        batch_shapes_cfg=cfg.batch_shapes_cfg)

# cfg.load_from = "/home/jayant/Projects/mmdetection/latest/work_dirs/yolov5_n-v61_syncbn_fast_1xb8-300e_cardboard/epoch_90.pth"
cfg.resume = True
cfg.val_evaluator = dict(
    type='mmdet.CocoMetric',
    proposal_nums=(100, 1, 10),
    ann_file=cfg.data_root + cfg.val_ann_file,
    metric='bbox')

cfg.train_cfg = dict(
    type='EpochBasedTrainLoop',
    max_epochs=50,
    val_interval=1)

cfg.work_dir = './work_dirs/yolov5_n-v61_syncbn_fast_1xb8-300e_box/'

In [5]:
cfg

Config (path: mmyolo/configs/yolov5/yolov5_n-v61_syncbn_fast_8xb16-300e_coco.py): {'default_scope': 'mmyolo', 'default_hooks': {'param_scheduler': {'type': 'YOLOv5ParamSchedulerHook', 'scheduler_type': 'linear', 'lr_factor': 0.01, 'max_epochs': 50}, 'checkpoint': {'type': 'CheckpointHook', 'interval': 10, 'save_best': 'auto', 'max_keep_ckpts': 3}}, 'env_cfg': {'cudnn_benchmark': True, 'mp_cfg': {'mp_start_method': 'fork', 'opencv_num_threads': 0}, 'dist_cfg': {'backend': 'nccl'}}, 'vis_backends': [{'type': 'LocalVisBackend'}], 'visualizer': {'type': 'mmdet.DetLocalVisualizer', 'vis_backends': [{'type': 'LocalVisBackend'}], 'name': 'visualizer'}, 'log_processor': {'type': 'LogProcessor', 'window_size': 50, 'by_epoch': True}, 'log_level': 'INFO', 'load_from': None, 'resume': True, 'backend_args': None, '_backend_args': None, 'tta_model': {'type': 'mmdet.DetTTAModel', 'tta_cfg': {'nms': {'type': 'nms', 'iou_threshold': 0.65}, 'max_per_img': 300}}, 'img_scales': [(640, 640), (320, 320), (9

In [6]:
from mmengine.runner import Runner

model = Runner.from_cfg(cfg)

model.train()

03/16 10:15:50 - mmengine - WARNING - Failed to search registry with scope "mmyolo" in the "log_processor" registry tree. As a workaround, the current "log_processor" registry in "mmengine" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmyolo" is a correct scope, or whether the registry is initialized.
03/16 10:15:50 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
    CUDA available: True
    MUSA available: False
    numpy_random_seed: 561654951
    GPU 0: NVIDIA GeForce RTX 3060 Laptop GPU
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0
    PyTorch: 2.0.0+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math 

KeyboardInterrupt: 